In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes 

from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests 

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans # import k-means from clustering stage

#!conda install -c conda-forge folium=0.5.0 --yes 

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


# Data preprocessing

In [2]:
# Get a list of cities of Washington State
tables = pd.read_html("https://en.wikipedia.org/wiki/List_of_cities_and_towns_in_Washington")

In [3]:
# Prepare dataframe of cities of Washington State
df_wa = tables[0]
df_wa = df_wa[['Name']]
df_wa.rename(columns={'Name': 'City'}, inplace=True)
df_wa = df_wa.reset_index(drop=True)
df_wa.columns = ['City']
df_wa['State']='Washington'
df_wa['Latitude']=''
df_wa['Longitude']=''
df_wa

/opt/conda/envs/Python36/lib/python3.6/site-packages/pandas/core/frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


,City,State,Latitude,Longitude
0,Aberdeen,Washington,,
1,Airway Heights,Washington,,
2,Algona,Washington,,
3,Anacortes,Washington,,
4,Arlington,Washington,,
5,Asotin,Washington,,
6,Auburn,Washington,,
7,Bainbridge Island,Washington,,
8,Battle Ground,Washington,,
9,Bellevue,Washington,,


### Getting latitude and longitude coordinates of each city using MapQuest API

In [4]:
# Add Mapquest credentials to run the following code
MapQ_key = '*******************************'
MapQ_secret = '*******************************'

In [5]:
for index, row in df_wa.iterrows():
    location = row['City'] + "," + row['State']
    url = 'https://www.mapquestapi.com/geocoding/v1/address?key={}&inFormat=kvp&outFormat=json&location={}&thumbMaps=false'.format(
    MapQ_key, 
    location)
    response = requests.get(url).json()
    df_wa.at[index,'Latitude'] = response['results'][0]['locations'][0]['latLng']['lat']
    df_wa.at[index,'Longitude'] = response['results'][0]['locations'][0]['latLng']['lng']
    
df_wa.to_csv('WashingtonCities.csv', sep=',',index=False)

df_wa

,City,State,Latitude,Longitude
0,Aberdeen,Washington,46.9755,-123.816
1,Airway Heights,Washington,47.643,-117.593
2,Algona,Washington,47.2791,-122.25
3,Anacortes,Washington,48.5054,-122.632
4,Arlington,Washington,48.1913,-122.126
5,Asotin,Washington,46.3406,-117.049
6,Auburn,Washington,47.3075,-122.226
7,Bainbridge Island,Washington,47.6431,-122.527
8,Battle Ground,Washington,45.7807,-122.548
9,Bellevue,Washington,47.6137,-122.191


In [6]:
#df = pd.read_csv('WashingtonCity.csv')

In [7]:
# Use geopy library to get the latitude and longitude values of Washington State
address = 'Washington State, USA'

geolocator = Nominatim(user_agent="northwest_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The latitude and longitude of Washington respectively are {}, {}.'.format(latitude, longitude))


The latitude and longitude of Washington respectively are 47.2868352, -120.2126139.


### Visualizing Map of Washington State

In [8]:
# create map of the Washington State using latitude and longitude values
map_wa = folium.Map(location=[latitude, longitude], zoom_start=7)

# add markers to map
for lat, lng, name in zip(df_wa['Latitude'], df_wa['Longitude'], df_wa['City']):
    label = '{}'.format(name)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_wa)  
    
map_wa

### Requesting Top Venues Using Foursquare API

In [9]:
# Use Foursquare developer credentials and add them here
CLIENT_ID = '*******************************************' # your Foursquare ID
CLIENT_SECRET = '*******************************************' # your Foursquare Secret
VERSION = '20200202' # Foursquare API version

In [10]:
# Check
city_latitude = df_wa.loc[0, 'Latitude'] # City latitude value
city_longitude = df_wa.loc[0, 'Longitude'] # City longitude value

city_name = df_wa.loc[0, 'City'] # Name
city_state = df_wa.loc[0, 'State'] # State

print('Latitude and longitude values of {}, {} are: {}, {}.'.format(city_name,
                                                               city_state,
                                                               city_latitude, 
                                                               city_longitude))

Latitude and longitude values of Aberdeen, Washington are: 46.97551, -123.815517.


In [11]:
# Get the top 100 venues within the default city radius
LIMIT = 100 # limit of number of venues returned by Foursquare API

# create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    city_latitude, 
    city_longitude, 
    LIMIT)

results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed13b8e882fc7001b9ca675'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'suggestedRadius': 10000,
  'headerLocation': 'Aberdeen',
  'headerFullLocation': 'Aberdeen',
  'headerLocationGranularity': 'city',
  'totalResults': 69,
  'suggestedBounds': {'ne': {'lat': 46.98438310725092,
    'lng': -123.7037284668247},
   'sw': {'lat': 46.943570747730774, 'lng': -123.91647086388492}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '52013263498ec644f18a2f77',
       'name': 'Rediviva Restaurant',
       'location': {'address': '118 E Wishkah',
        'lat': 46.97482644098638,
        'lng': -123.81710478759817,
        'labeled

In [12]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [13]:
# Clean the data and structure it as a dataframe
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Rediviva Restaurant,New American Restaurant,46.974826,-123.817105
1,Oceana Spa,Spa,46.971251,-123.822982
2,Amore' Italian Restaurant,Italian Restaurant,46.973007,-123.817937
3,Dutch Bros Coffee,Café,46.977405,-123.808403
4,Sucher & Sons Star Wars Shop,Toy / Game Store,46.976254,-123.813901


In [14]:
nearby_venues.to_csv('NearbyVenues.csv', sep=',',index=False)

In [15]:
#nearby_venues = pd.read_csv('NearbyVenues.csv')

In [16]:
# Create a function to repeat the same process to all cities
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [17]:
# Run the function on each city and store in new dataframe
washington_venues = getNearbyVenues(names=df_wa['City'],
                                   latitudes=df_wa['Latitude'],
                                   longitudes=df_wa['Longitude'])

Aberdeen
Airway Heights
Algona
Anacortes
Arlington
Asotin
Auburn
Bainbridge Island
Battle Ground
Bellevue
Bellingham
Benton City
Bingen
Black Diamond
Blaine
Bonney Lake
Bothell
Bremerton
Brewster
Bridgeport
Brier
Buckley
Burien
Burlington
Camas
Carnation
Cashmere
Castle Rock
Centralia
Chehalis
Chelan
Cheney
Chewelah
Clarkston
Cle Elum
Clyde Hill
Colfax
College Place
Colville
Connell
Cosmopolis
Covington
Davenport
Dayton
Deer Park
Des Moines
DuPont
Duvall
East Wenatchee
Edgewood
Edmonds
Electric City
Ellensburg
Elma
Entiat
Enumclaw
Ephrata
Everett
Everson
Federal Way
Ferndale
Fife
Fircrest
Forks
George
Gig Harbor
Gold Bar
Goldendale
Grand Coulee
Grandview
Granger
Granite Falls
Harrington
Hoquiam
Ilwaco
Issaquah
Kahlotus
Kalama
Kelso
Kenmore
Kennewick
Kent
Kettle Falls
Kirkland
Kittitas
La Center
Lacey
Lake Forest Park
Lake Stevens
Lakewood
Langley
Leavenworth
Liberty Lake
Long Beach
Longview
Lynden
Lynnwood
Mabton
Maple Valley
Marysville
Mattawa
McCleary
Medical Lake
Medina
Mercer Islan

In [18]:
washington_venues.to_csv('WashingtonVenues.csv', sep=',',index=False)

In [19]:
# washington_venues = pd.read_csv('WashingtonVenues.csv')

In [20]:
print(washington_venues.shape)
washington_venues.head()

(2740, 7)


,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Aberdeen,46.97551,-123.815517,Rediviva Restaurant,46.974826,-123.817105,New American Restaurant
1,Aberdeen,46.97551,-123.815517,Sucher & Sons Star Wars Shop,46.976254,-123.813901,Toy / Game Store
2,Aberdeen,46.97551,-123.815517,Billy's Bar and Grill,46.975505,-123.813540,Bar
3,Aberdeen,46.97551,-123.815517,Amore' Italian Restaurant,46.973007,-123.817937,Italian Restaurant
4,Aberdeen,46.97551,-123.815517,Breakwater Seafood And Chowder,46.975644,-123.811814,Seafood Restaurant


In [21]:
# Tally up the total venues per city
grouped = washington_venues.groupby('City').count()
print('Original count of cities: {}'.format(len(grouped.index)))

# Drop cities with inadequate amount of venue data (they skew the clustering results)
grouped = grouped[grouped.Venue < 10]
list(grouped.index.values)
northwest_venues = washington_venues[~washington_venues['City'].isin(list(grouped.index.values))]
grouped2 = northwest_venues.groupby('City').count()

print('Count of cities with more than 10 venues: {}'.format(len(grouped2.index)))

Original count of cities: 208
Count of cities with more than 10 venues: 82


In [22]:
print('There are {} unique categories.'.format(len(washington_venues['Venue Category'].unique())))

There are 301 unique categories.


### One Hot Encoding

In [23]:
washington_onehot = pd.get_dummies(washington_venues[['Venue Category']], prefix="", prefix_sep="")

# Add city column back to dataframe
washington_onehot['City'] = washington_venues['City'] 

# move city column to the first column
fixed_columns = [washington_onehot.columns[-1]] + list(washington_onehot.columns[:-1])
washington_onehot = washington_onehot[fixed_columns]

# Check size of new dataframe
washington_onehot.shape

(2740, 301)

In [24]:
washington_grouped = washington_onehot.groupby('City').mean().reset_index()
washington_grouped

,City,Yoga Studio,ATM,Accessories Store,Adult Boutique,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Arcade,Art Gallery,Art Studio,Arts & Crafts Store,Asian Restaurant,Assisted Living,Athletics & Sports,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bistro,Board Shop,Boat Rental,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Carpet Store,Casino,Cheese Shop,Chinese Restaurant,Chiropractor,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cosmetics Shop,Credit Union,Cupcake Shop,Dance Studio,Deli / Bodega,Dentist's Office,Department Store,Dessert Shop,Diner,Disc Golf,Discount Store,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Eastern European Restaurant,Electronics Store,Emergency Room,English Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Financial or Legal Service,Fireworks Store,Fish & Chips Shop,Fish Market,Fishing Spot,Flea Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Service,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,High School,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunting Supply,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Inn,Insurance Office,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Knitting Store,Korean Restaurant,Lake,Latin American Restaurant,Laundromat,Lawyer,Lebanese Restaurant,Library,Lighthouse,Liquor Store,Locksmith,Lounge,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Motorsports Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Nightlife Spot,Noodle House,Optical Shop,Organic Grocery,Other Repair Shop,Outdoor Sculpture,Outdoor Supply Store,Paella Restaurant,Paper / Office Supplies Store,Park,Pawn Shop,Performing Arts Venue,Persian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Pier,Pilates Studio,Pizza Place,Platform,Playground,Plaza,Poke Place,Pool,Pool Hall,Post Office,Print Shop,Pub,RV Park,Recreation Center,Rental Car Location,Residential Building (Apartment / Condo),Resort,Restaurant,River,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Science Museum,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Shopping Plaza,Skate Park,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,Social Club,Soup Place,Southern / Soul Food Restaurant,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Stadium,State / Provincial Park,Steakhouse,Supermarket,Supplement Shop,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Theme Park,Thrift / Vintage Store,Tiki Bar,Tour Provider,Tourist Information Center,Toy / Game Store,Track,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vid

In [25]:
# New dataframe size
washington_grouped.shape

(208, 301)

### Top 5 Each City Avenues

In [26]:
num_top_venues = 5

for city in washington_grouped['City']:
    print("----"+city+"----")
    temp = washington_grouped[washington_grouped['City'] == city].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Aberdeen----
                  venue  freq
0  Fast Food Restaurant  0.11
1           Coffee Shop  0.11
2    Italian Restaurant  0.06
3                 Hotel  0.06
4                   Bar  0.06


----Airway Heights----
                 venue  freq
0          Video Store  0.09
1   Chinese Restaurant  0.09
2        Grocery Store  0.09
3                 Park  0.09
4  Japanese Restaurant  0.09


----Algona----
               venue  freq
0               Food  0.17
1      Moving Target  0.17
2                Bar  0.17
3  Convenience Store  0.17
4        Pizza Place  0.17


----Anacortes----
               venue  freq
0       Home Service  0.33
1  Martial Arts Dojo  0.33
2       Soccer Field  0.33
3        Yoga Studio  0.00
4  Other Repair Shop  0.00


----Arlington----
                     venue  freq
0          Automotive Shop  0.17
1      American Restaurant  0.17
2      Arts & Crafts Store  0.17
3  New American Restaurant  0.17
4           Farmers Market  0.17


----Asotin----
        

In [27]:
# Function to sort venues in decscending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [28]:
# Create a dataframe with top 5 venues for each city
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# Create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# Create a new dataframe
cities_venues_sorted = pd.DataFrame(columns=columns)
cities_venues_sorted['City'] = washington_grouped['City']

for ind in np.arange(washington_grouped.shape[0]):
    cities_venues_sorted.iloc[ind, 1:] = return_most_common_venues(washington_grouped.iloc[ind, :], num_top_venues)

cities_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Aberdeen,Fast Food Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Music Venue
1,Airway Heights,Pizza Place,Video Store,Café,Bar,Thai Restaurant
2,Algona,Business Service,Pizza Place,Convenience Store,Food,Moving Target
3,Anacortes,Martial Arts Dojo,Soccer Field,Home Service,Women's Store,Farm
4,Arlington,Farmers Market,Diner,American Restaurant,Arts & Crafts Store,Automotive Shop


### KMeans Clustering

In [29]:
# Run K-means to break up into clusters
kclusters = 6

washington_grouped_clustering = washington_grouped.drop('City', 1)

# Run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(washington_grouped_clustering)

# Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:300]

array([3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 1, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3,
       0, 3, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 0,
       0, 2, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 0, 3, 3, 3, 3, 5, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 5,
       3, 3, 0, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 0, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 3, 4, 3, 3, 0, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 4, 3, 1, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3], dtype=int32)

In [30]:
cities_venues_sorted.columns

Index(['City', '1st Most Common Venue', '2nd Most Common Venue',
       '3rd Most Common Venue', '4th Most Common Venue',
       '5th Most Common Venue'],
      dtype='object')

In [31]:
cities_venues_sorted

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Aberdeen,Fast Food Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Music Venue
1,Airway Heights,Pizza Place,Video Store,Café,Bar,Thai Restaurant
2,Algona,Business Service,Pizza Place,Convenience Store,Food,Moving Target
3,Anacortes,Martial Arts Dojo,Soccer Field,Home Service,Women's Store,Farm
4,Arlington,Farmers Market,Diner,American Restaurant,Arts & Crafts Store,Automotive Shop
5,Asotin,Convenience Store,Women's Store,Fishing Spot,Farm,Farmers Market
6,Auburn,Coffee Shop,Café,Bank,Bar,American Restaurant
7,Bainbridge Island,Gym / Fitness Center,Bakery,Fish Market,Farm,Farmers Market
8,Battle Ground,Pizza Place,Coffee Shop,Ice Cream Shop,Flea Market,Café
9,Bellevue,Coffee Shop,Hotel,Women's Store,Bank,Steakhouse


In [32]:
# Create dataframe that includes the cluster and top 5 venues

# Add clustering labels
cities_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

washington_merged = df_wa

# Merge washington_grouped with northwest_data to add latitude/longitude for each city
washington_merged = washington_merged.join(cities_venues_sorted.set_index('City'), on='City')

# Drop cities with no venue data
washington_merged = washington_merged.dropna()

washington_merged

,City,State,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Aberdeen,Washington,46.9755,-123.816,3.0,Fast Food Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Music Venue
1,Airway Heights,Washington,47.643,-117.593,3.0,Pizza Place,Video Store,Café,Bar,Thai Restaurant
2,Algona,Washington,47.2791,-122.25,3.0,Business Service,Pizza Place,Convenience Store,Food,Moving Target
3,Anacortes,Washington,48.5054,-122.632,3.0,Martial Arts Dojo,Soccer Field,Home Service,Women's Store,Farm
4,Arlington,Washington,48.1913,-122.126,3.0,Farmers Market,Diner,American Restaurant,Arts & Crafts Store,Automotive Shop
5,Asotin,Washington,46.3406,-117.049,4.0,Convenience Store,Women's Store,Fishing Spot,Farm,Farmers Market
6,Auburn,Washington,47.3075,-122.226,3.0,Coffee Shop,Café,Bank,Bar,American Restaurant
7,Bainbridge Island,Washington,47.6431,-122.527,3.0,Gym / Fitness Center,Bakery,Fish Market,Farm,Farmers Market
8,Battle Ground,Washington,45.7807,-122.548,3.0,Pizza Place,Coffee Shop,Ice Cream Shop,Flea Market,Café
9,Bellevue,Washington,47.6137,-122.191,3.0,Coffee Shop,Hotel,Women's Store,Bank,Steakhouse


### Mapping Result of KMeans Cluster

In [33]:
# Create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=7)

# Set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# Add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(washington_merged['Latitude'], washington_merged['Longitude'], washington_merged['City'], washington_merged['Cluster Labels']):
    cluster = int(cluster)
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Project Result

Cluster 0: It has 19 cities with mainly parks and farms avenue. So I think these cities have many wide open areas and suitable for people who love nature. 

In [34]:
# Cluster 0
washington_merged.loc[washington_merged['Cluster Labels'] == 0, washington_merged.columns[[0] + [1] + list(range(5, washington_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
31,Cheney,Washington,Business Service,Park,Bar,Volleyball Court,Women's Store
37,College Place,Washington,Park,Photography Studio,Construction & Landscaping,Fish Market,Farm
39,Connell,Washington,Post Office,Gym / Fitness Center,Pizza Place,Movie Theater,Park
46,DuPont,Washington,History Museum,Business Service,Park,Burger Joint,Farm
64,George,Washington,Food Truck,State / Provincial Park,Sandwich Place,Park,Fireworks Store
67,Goldendale,Washington,Restaurant,Brewery,Pizza Place,Bar,RV Park
68,Grand Coulee,Washington,Ice Cream Shop,Pizza Place,Park,Grocery Store,Fondue Restaurant
70,Granger,Washington,Park,Women's Store,Fish & Chips Shop,Falafel Restaurant,Farm
72,Harrington,Washington,Convenience Store,Park,Women's Store,Fish Market,Farm
92,Liberty Lake,Washington,Pool,Art Gallery,Park,Business Service,Food


Cluster 1: There are only four cities listed in cluster 1

In [35]:
# Cluster 1
washington_merged.loc[washington_merged['Cluster Labels'] == 1, washington_merged.columns[[0] + [1] + list(range(5, washington_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
32,Chewelah,Washington,Pizza Place,Home Service,Women's Store,Fish Market,Falafel Restaurant
49,Edgewood,Washington,Coffee Shop,Gym,Convenience Store,Pizza Place,Home Service
171,Soap Lake,Washington,Coffee Shop,Women's Store,Fishing Spot,Farm,Farmers Market
193,University Place,Washington,Spa,Coffee Shop,Women's Store,Fish Market,Farm


Cluster 2: Based on the KMeans Clustering method, Grandview City is the most unique place in the Washington State

In [36]:
# Cluster 2
washington_merged.loc[washington_merged['Cluster Labels'] == 2, washington_merged.columns[[0] + [1] + list(range(5, washington_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
69,Grandview,Washington,Nightlife Spot,Women's Store,Event Space,Farm,Farmers Market


Cluster 3: This cluster has many cities on the list. The most striking feature is that Seattle belongs to this cluster. So, I think these cities on the list have many similarities to Seattle and might be people's alternative to find home where the city are less populated than Seattle but have Seattle vibes. More over, the cities on this cluster have many entertainment avenues as the top five avenues such bar, restaurant, cafe, coffe shop, and many more which is good for bussiness.   

In [37]:
# Cluster 3
washington_merged.loc[washington_merged['Cluster Labels'] == 3, washington_merged.columns[[0] + [1] + list(range(5, washington_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Aberdeen,Washington,Fast Food Restaurant,Coffee Shop,Pizza Place,Italian Restaurant,Music Venue
1,Airway Heights,Washington,Pizza Place,Video Store,Café,Bar,Thai Restaurant
2,Algona,Washington,Business Service,Pizza Place,Convenience Store,Food,Moving Target
3,Anacortes,Washington,Martial Arts Dojo,Soccer Field,Home Service,Women's Store,Farm
4,Arlington,Washington,Farmers Market,Diner,American Restaurant,Arts & Crafts Store,Automotive Shop
6,Auburn,Washington,Coffee Shop,Café,Bank,Bar,American Restaurant
7,Bainbridge Island,Washington,Gym / Fitness Center,Bakery,Fish Market,Farm,Farmers Market
8,Battle Ground,Washington,Pizza Place,Coffee Shop,Ice Cream Shop,Flea Market,Café
9,Bellevue,Washington,Coffee Shop,Hotel,Women's Store,Bank,Steakhouse
10,Bellingham,Washington,Coffee Shop,Café,Bar,American Restaurant,Breakfast Spot


Cluster 4: It has only four cities listed on this cluster. The top avenues here are fishing spot and farmers market. I think this cluster has similar condition with cluster 0. 

In [38]:
# Cluster 4
washington_merged.loc[washington_merged['Cluster Labels'] == 4, washington_merged.columns[[0] + [1] + list(range(5, washington_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
5,Asotin,Washington,Convenience Store,Women's Store,Fishing Spot,Farm,Farmers Market
15,Bonney Lake,Washington,Bar,Women's Store,Fishing Spot,Farmers Market,Fast Food Restaurant
173,Spangle,Washington,Bar,American Restaurant,Women's Store,Fishing Spot,Farmers Market
191,Tumwater,Washington,Convenience Store,Bar,Women's Store,Fishing Spot,Farmers Market


Cluster 5: This cluster has unique avenues on the top 5 avenues such as candy store and falafel restaurant making them unique in this dataset

In [39]:
# Cluster 5
washington_merged.loc[washington_merged['Cluster Labels'] == 5, washington_merged.columns[[0] + [1] + list(range(5, washington_merged.shape[1]))]]

,City,State,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
97,Mabton,Washington,Business Service,Women's Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant
134,Pacific,Washington,Business Service,Gym,Candy Store,Women's Store,Farmers Market


## Conclusion

We first webscraped data from Wikipedia to get list of cities in Washington State. There were 212 cities on the listthat we needed to find their location data which were latitude and longitude coordinates of each city. To get those information, we used MapQuest API. After that, we used FourSquare API to get FourSquare query of venues, there were 2740 venues in the Washington State including 301 unique category venues. To solve the probrlem, we used KMeans clustering in order to group the cities into 6 clusters. However there are only four data that are reliable. This project needs to be improved whether using more reliable dataset or clustering method. Based on this project, we sure can answer the main problem stated in the beginning of this project that is "are there nearby city around Washington state that similar to Seattle to live in?". And the answer are listed on cluster 3.